In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [6]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [5]:
# set up data loader
train_gen = SequenceGenLong(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGenLong(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalizedLong(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalizedLong(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [74]:
conv = Sequential(ConvBatchRelu1D(1,2,3),
                  Dropout(0.5),
                  ConvBatchRelu1D(2,4,3),
                  Dropout(0.5),
                  ConvBatchRelu1D(4,8,3,stride=2),
                  Dropout(0.5),
                  ConvBatchRelu1D(8,16,3,stride=2),
                  Dropout(0.5))

In [75]:
rnn = GRU(16,32,2,batch_first=True,dropout=0.5,bidirectional=True)

In [76]:
linear = Sequential(Linear(64,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [77]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized(conv,rnn,linear).to('cuda:0')

In [78]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [79]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [80]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip)

epoch:0, train_loss:0.894094851319654, val_loss:0.9795907139778137
epoch:1, train_loss:0.8379507854503672, val_loss:0.9550752639770508
epoch:2, train_loss:0.8334924305658884, val_loss:0.9765797257423401
Training completed in 126.15942144393921s


In [81]:
model_nor = fit(1, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip)

epoch:0, train_loss:0.825160918705204, val_loss:0.9312103390693665
Training completed in 42.08376479148865s


In [82]:
model = CNN_RNN2seq_logNormalized(conv,rnn,linear).to('cuda:0')

In [83]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [84]:
loss_func = loss_func_generator(SMAPE)

In [85]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:39.20021782217866, val_loss:40.480987548828125
epoch:1, train_loss:39.08747937493991, val_loss:40.22978591918945
epoch:2, train_loss:39.10863829400255, val_loss:40.04621124267578
epoch:3, train_loss:39.03249246567642, val_loss:40.18659973144531
epoch:4, train_loss:39.03590043715245, val_loss:39.50408172607422
Training completed in 231.5190281867981s
